In [1]:
!pip install datasets
!pip install transformers
!pip install seqeval
!pip install accelerate -U
!pip install transformers[torch]
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=9b3416ba195bb6fbca969d30cf5780bf8aeb3b74fe71e4afdcf1f2f3f0b053e7
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.0 MB/s eta 0:00:00


In [2]:
#from huggingface_hub import notebook_login

#notebook_login()

In [3]:
from datasets import load_dataset# what is finetuning!? # sep cls in ner ?
import numpy as np 
from datasets import Dataset
import ast
import evaluate
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer
from datasets import load_metric
from transformers import DataCollatorForTokenClassification
import os
os.environ['WANDB_DISABLED'] = 'true'
# какая-то ошибка в ner список на самом деле в формате str
tr = load_dataset('csv', data_files='../input/newfolds/newtrainfold_0.csv').remove_columns('Unnamed: 0')
tr = tr.shuffle(seed= 14)
ts = load_dataset('csv', data_files='../input/newfolds/newtestfold_0.csv').remove_columns('Unnamed: 0')


id2label = {0 : 'O', 1 : 'PAT', 2 : 'PHY', 3 : 'PAT-START', 4 : 'PHY-START'}
label2id = {'O' : 0, 'PAT' : 1, 'PHY' : 2, 'PAT-START' : 3, 'PHY-START' : 4}
label_list = ['O', 'PAT', 'PHY', 'PAT-START', 'PHY-START']

def from_str_to_list(data):
    out = []
    for i in data:
        out.append(ast.literal_eval(i))
    return out

train = tr['train']
test = ts['train']

trner = train['ner']
trwords = train['words']

tsner = test['ner']
tswords = test['words']

train = train.remove_columns('ner')
train = train.add_column('ner',from_str_to_list(trner))
train = train.remove_columns('words')
train = train.add_column('words',from_str_to_list(trwords))

test = test.remove_columns('ner')
test = test.add_column('ner',from_str_to_list(tsner))
test = test.remove_columns('words')
test = test.add_column('words',from_str_to_list(tswords))

#tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny-sentiment-balanced')
tokenizer = AutoTokenizer.from_pretrained('IlyaGusev/rubert_ext_sum_gazeta')
#tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased') # how to split ner text

max_length = 512
def tokenize_and_align_labels(examples): # trunc texts and labels
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["words"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(label2id[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


tokenized_train = train.map(tokenize_and_align_labels, batched=True)
tokenized_test = test.map(tokenize_and_align_labels, batched=True)


metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}


#'IlyaGusev/rubert_ext_sum_gazeta'
model = AutoModelForTokenClassification.from_pretrained('IlyaGusev/rubert_ext_sum_gazeta', num_labels=5,
                                                        label2id=label2id,
                                                        id2label = id2label,
                                                        ignore_mismatched_sizes=True)

data_collator = DataCollatorForTokenClassification(tokenizer)

task = "ner"
args = TrainingArguments(
    output_dir = 'model',
    evaluation_strategy = "epoch",
    learning_rate = 4e-5,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    num_train_epochs = 2,
    weight_decay = 1e-5,
    #push_to_hub = True,
)

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_test,
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)



/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e3b6df29dbcc1ec5/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


  0%|          | 0/1 [00:00<?, ?it/s]

...

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at IlyaGusev/rubert_ext_sum_gazeta and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [4]:
trainer.train()
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.276544,0.628399,0.634146,0.631259,0.914604
2,No log,0.238646,0.663934,0.740854,0.700288,0.918626


/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAT-START seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PHY-START seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PHY seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.23864641785621643,
 'eval_precision': 0.6639344262295082,
 'eval_recall': 0.7408536585365854,
 'eval_f1': 0.7002881844380403,
 'eval_accuracy': 0.9186262376237624,
 'eval_runtime': 3.3088,
 'eval_samples_per_second': 49.867,
 'eval_steps_per_second': 12.693,
 'epoch': 2.0}

In [5]:
trainer.save_model('model1') # иногда модель сохраняется некорректно, при импорте дает ошибку

# Подсчет метрик вручную

In [6]:
pred = trainer.predict(tokenized_test)
preds = np.argmax(pred.predictions, axis=-1)
target = pred.label_ids

In [7]:
n = 0
cut_target = []
for i in target:
    while n < len(i):
        if n == 0:
            n += 1
            continue
        if i[n] == -100:
            break
        n += 1
    cut_target.append(i[:n])
    n = 0
            

In [8]:
cut_preds = []
i = 0
while i < len(preds):
    cut_preds.append(preds[i][:len(cut_target[i])])
    i += 1

In [9]:
# recode
f_preds = []
f_targets = []
temp = []
for i in cut_preds:
    for j in i:
        a = 0
        temp.append(id2label[j])
    f_preds.append(temp)
    temp = []
for i in cut_target:
    for j in i:
        a = 0
        if j == -100:
            temp.append(id2label[0])
        else:temp.append(id2label[j])
    f_targets.append(temp)
    temp = []

In [10]:
i = 0
while i < len(cut_preds):
    if len(cut_preds) != len(cut_target):
        print (i)
    i += 1

In [11]:
seqeval = evaluate.load('seqeval')
results = seqeval.compute(predictions=f_preds, references=f_targets)

In [12]:
results

{'AT': {'precision': 0.5140845070422535,
  'recall': 0.6576576576576577,
  'f1': 0.5770750988142292,
  'number': 111},
 'HY': {'precision': 0.5, 'recall': 0.5, 'f1': 0.5, 'number': 42},
 'START': {'precision': 0.8186813186813187,
  'recall': 0.8514285714285714,
  'f1': 0.8347338935574229,
  'number': 175},
 'overall_precision': 0.6639344262295082,
 'overall_recall': 0.7408536585365854,
 'overall_f1': 0.7002881844380403,
 'overall_accuracy': 0.9225787459523108}

# Метрики выгруженной модели

In [17]:
from transformers import AutoTokenizer, BertForTokenClassification
from datasets import load_dataset
import ast
model_name = "model1"
tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=True)
model = BertForTokenClassification.from_pretrained(model_name, local_files_only=True)

In [18]:
def predict(sample):
    sample = ' '.join(sample)
    encoding = tokenizer(sample, return_tensors="pt", padding=True)
    outputs = model(**encoding)
    logits = outputs.logits
    
    return logits.argmax(-1) # pred

In [19]:
predictions = []
for i in test['words']:
    predictions.append(predict(i))


In [48]:
# перекодировка предикта из индексов в текст
i = 0
load_preds = []
temp = []
while i < len(predictions):
    for j in predictions[i][0]:
        temp.append(id2label[int(j)])
    load_preds.append(temp[:-1])
    temp = []
    i += 1

In [53]:
seqeval.compute(predictions=load_preds, references=f_targets) # matches the target like trainer model

{'AT': {'precision': 0.5140845070422535,
  'recall': 0.6576576576576577,
  'f1': 0.5770750988142292,
  'number': 111},
 'HY': {'precision': 0.5, 'recall': 0.5, 'f1': 0.5, 'number': 42},
 'START': {'precision': 0.8186813186813187,
  'recall': 0.8514285714285714,
  'f1': 0.8347338935574229,
  'number': 175},
 'overall_precision': 0.6639344262295082,
 'overall_recall': 0.7408536585365854,
 'overall_f1': 0.7002881844380403,
 'overall_accuracy': 0.9225787459523108}

In [54]:
seqeval.compute(predictions=load_preds, references=f_preds) # mathces trainer model

{'AT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 142},
 'HY': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 42},
 'START': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 182},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}